`Insert title and authors here`

# 1. Introduction 

`Write about the context of the processing task to be analyzed in this project (ie what is SEVN, what is data used for in astrophysics context, etc...)`

# 2. Datasets

`Write about the structure of the datasets here. Add diagrams/tables if useful`

# 3. Cluster

`Describe briefly the setup of cluster and file system. Add diagrams if useful`

# 4. Data Processing 

## 4.1 Naive method

`Method used for LCP-B before knowing Dask. Report performance`

## 4.2 Optimized method (bags? foldby? delay? etc...)

`Optimize naive method as much as possible. Again report performance`

## 4.3 (optional) Cluster behavior

`Try to understand the way dask and NFS handle data transfer, processing, etc... Especially considering that the NFS server is assigned some processing tasks and the scheduler is assigned processing tasks.`

# 5. Benchmarking

`Benchmark the best algorithm by collecting statistics on the time performance at varying numbers of threads, workers, and partitions`